In [12]:
import numpy as np
import pandas as pd
from FoodSeg103_data_handler import *

Load the images as batches

In [13]:
img_nums = [n for n in range(0, 4900, 500)] #2750 #1234 #2314
imgs, gts = load_food_batch(img_nums)
num_imgs = len(imgs)
print(f"Total images: {num_imgs}")

Total images: 10


Check and filter for resolution

In [14]:
resol = {im_num: np.prod(im.shape[:2]) for im_num, im in imgs.items()}

# Check for images with resolution over a certain threshold
thresh = 200_000
print("Resolutions:")
for im_num, r in resol.items(): print(f"{im_num:4d}: {r:9,d} pixels{' (!)' if r > thresh else ''}")

Resolutions:
   0:   196,608 pixels
 500:   307,200 pixels (!)
1000:   196,608 pixels
1500:   133,660 pixels
2000:   172,544 pixels
2500:   196,608 pixels
3000:   174,080 pixels
3500:   196,608 pixels
4000:   196,608 pixels
4500: 3,145,728 pixels (!)


In [15]:
# Filter out those images
imgs = {im_num: im for im_num, im in imgs.items() if resol[im_num] < thresh}
num_imgs = len(imgs)
print(f"Images with resolution under {thresh:,d}: {num_imgs}")
resol_new = {im_num: np.prod(im.shape[:2]) for im_num, im in imgs.items()}
for im_num, r in resol_new.items(): print(f"{im_num:4d}: {r:7,d} pixels")

Images with resolution under 200,000: 8
   0: 196,608 pixels
1000: 196,608 pixels
1500: 133,660 pixels
2000: 172,544 pixels
2500: 196,608 pixels
3000: 174,080 pixels
3500: 196,608 pixels
4000: 196,608 pixels


Define parameters

In [16]:
# Which scribbles to use
bins = [0.05, 0.3, 1.5]
suff = ["w3"] #standing for width 3

# Which prediction models to use
convpaint_layers = [[0], [0,2]]
convpaint_scalings = [[1], [1,2], [1,2,4], [1,2,4,8]]
use_dino = True
use_ilastik = True

# Where to find and save the data
folder_path = "./FoodSeg103_results/data/run02"

Loop and analyse

In [17]:
df = pd.DataFrame(columns=['img_num', 'prediction type', 'scribbles mode', 'scribbles bin', 'suffix', 'max_class_pix_gt', 'min_class_pix_gt', 'pix_labelled', 'max_class_pix_labelled', 'min_class_pix_labelled', 'pix_in_img', 'perc. labelled', 'accuracy', 'scribbles', 'prediction'])

for img_num, img in imgs.items():
    ground_truth = gts[img_num]
    for bin in bins:
        for s in suff:
            print(f"IMG {img_num}: bin {bin}, suff {s}")
            for layers in convpaint_layers:
                for scalings in convpaint_scalings:
                    print(f"ConvPaint: layers {layers}, scalings {scalings}")
                    pred_tag = generate_convpaint_tag(layers, scalings)
                    res = analyse_food_single_file(ground_truth, folder_path, img_num, mode="all", bin=bin, suff=s, pred_tag=pred_tag, show_res=False, image=None)
                    df = pd.concat([df, res], ignore_index=True)
            if use_ilastik:
                print("Ilastik")
                res = analyse_food_single_file(ground_truth, folder_path, img_num, mode="all", bin=bin, suff=s, pred_tag="ilastik", show_res=False, image=None)
                df = pd.concat([df, res], ignore_index=True)
            if use_dino:
                print("DINO")
                res = analyse_food_single_file(ground_truth, folder_path, img_num, mode="all", bin=bin, suff=s, pred_tag="dino", show_res=False, image=None)
                df = pd.concat([df, res], ignore_index=True)
            print("\n")

IMG 0: bin 0.05, suff w3
ConvPaint: layers [0], scalings [1]
ConvPaint: layers [0], scalings [1, 2]
ConvPaint: layers [0], scalings [1, 2, 4]
ConvPaint: layers [0], scalings [1, 2, 4, 8]
ConvPaint: layers [0, 2], scalings [1]
ConvPaint: layers [0, 2], scalings [1, 2]
ConvPaint: layers [0, 2], scalings [1, 2, 4]


C:\Users\roman\AppData\Local\Temp\ipykernel_964\3834574521.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, res], ignore_index=True)


ConvPaint: layers [0, 2], scalings [1, 2, 4, 8]
Ilastik
DINO


IMG 0: bin 0.3, suff w3
ConvPaint: layers [0], scalings [1]
ConvPaint: layers [0], scalings [1, 2]
ConvPaint: layers [0], scalings [1, 2, 4]
ConvPaint: layers [0], scalings [1, 2, 4, 8]
ConvPaint: layers [0, 2], scalings [1]
ConvPaint: layers [0, 2], scalings [1, 2]
ConvPaint: layers [0, 2], scalings [1, 2, 4]
ConvPaint: layers [0, 2], scalings [1, 2, 4, 8]
Ilastik
DINO


IMG 0: bin 1.5, suff w3
ConvPaint: layers [0], scalings [1]
ConvPaint: layers [0], scalings [1, 2]
ConvPaint: layers [0], scalings [1, 2, 4]
ConvPaint: layers [0], scalings [1, 2, 4, 8]
ConvPaint: layers [0, 2], scalings [1]
ConvPaint: layers [0, 2], scalings [1, 2]
ConvPaint: layers [0, 2], scalings [1, 2, 4]
ConvPaint: layers [0, 2], scalings [1, 2, 4, 8]
Ilastik
DINO


IMG 1000: bin 0.05, suff w3
ConvPaint: layers [0], scalings [1]
ConvPaint: layers [0], scalings [1, 2]
ConvPaint: layers [0], scalings [1, 2, 4]
ConvPaint: layers [0], scalings [1, 2, 4, 

In [18]:
df.groupby(['prediction type'])[['accuracy']].mean()

,accuracy
prediction type,
convpaint_l-0-2_s-1,0.756027
convpaint_l-0-2_s-1-2,0.749196
convpaint_l-0-2_s-1-2-4,0.723087
convpaint_l-0-2_s-1-2-4-8,0.683511
convpaint_l-0_s-1,0.774896
convpaint_l-0_s-1-2,0.768050
convpaint_l-0_s-1-2-4,0.742269
convpaint_l-0_s-1-2-4-8,0.722415
dino,0.913746


Save the data frame in a csv file:

In [19]:
from datetime import datetime
time_stamp = datetime.now().strftime("%y%m%d%H%M%S")
file_name = f"test_labels_vs_acc_{time_stamp}.csv"
df.to_csv(file_name, index=False)